# Exploratory Data Analysis

blah blah blah

### Load the data

In [1]:
from sqlalchemy import create_engine, Column, String, Integer, DATE
from sqlalchemy.orm import sessionmaker

from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import func


from itertools import product
from collections import defaultdict

import pandas as pd
import numpy as np
import json

import spacy

import sys

In [2]:
with open('../../postgres.json') as pg_info:
    pg_json = json.load(pg_info)
    pg_username = pg_json['pg_username']
    pg_password = pg_json['pg_password']
    pg_ip = pg_json['pg_ip']

engine = create_engine(f'postgres://{pg_username}:{pg_password}@{pg_ip}:5432')

In [3]:
Base = declarative_base()

# the article class is how sqlalchemy treats the objects of a row
class Articles(Base):
    __tablename__ = 'arxiv_raw'
    
    id = Column(String, primary_key=True)
    created = Column(DATE)
    setspec = Column(String)
    title = Column(String)
    abstract = Column(String)
    
Session = sessionmaker(bind=engine)


In [4]:
#get a new session into the database
session = Session()

#use the session to query the database
query = session.query(Articles)


In [37]:
from sqlalchemy.sql.expression import func
    

query = session.query(Articles).limit(10)

In [38]:
df = pd.read_sql(query.statement, query.session.bind)

In [39]:
df

,id,created,setspec,title,abstract
0,1707.06238,2017-07-19,physics:astro-ph,Orbiting a binary: SPHERE characterisation of ...,In this paper we present the results of the ...
1,1707.06239,2017-07-19,physics:astro-ph,An astrometric search method for individually ...,Gravitational waves (GWs) cause the apparent...
2,1707.06240,2017-07-19,math,Second-order Bounds of Gaussian Kernel-based F...,Guaranteeing stability of a designed control...
3,1707.06241,2017-07-19,physics:astro-ph,MUSE sneaks a peek at extreme ram-pressure eve...,We present the MUSE observations of UGC 6697...
4,1707.06242,2017-07-19,physics:hep-th,Green-Schwarz Automorphisms and 6D SCFTs,All known interacting 6D superconformal fiel...
5,1707.06243,2017-07-19,math,Rigorous free fermion entanglement renormaliza...,We construct entanglement renormalization sc...
6,math/9302204,1993-02-02,math,The Complete Continuity Property and Finite Di...,A Banach space $\X$ has the complete continu...
7,1707.06245,2017-07-19,physics:hep-th,Tweaking one-loop determinants in AdS$_3$,We revisit the subject of one-loop determina...
8,1707.06246,2017-07-19,physics:astro-ph,Extremal Kerr-Newman black holes with extremel...,The recently proved `no short hair' theorem ...
9,1707.06247,2017-07-19,cs,On the Economics of Ransomware,While recognized as a theoretical and practi...


In [99]:
#read the query into a dataframe
%time df = pd.read_sql(query.statement, query.session.connection())

CPU times: user 13.4 s, sys: 29 s, total: 42.4 s
Wall time: 4min 20s


In [100]:
df.shape

(1407628, 5)

In [101]:
type(df.title[0])

str

In [170]:
nlp = spacy.load('en_core_web_sm')  

In [178]:
mini_size = 10000
minidf = df.iloc[:mini_size].copy()
docdf = minidf
%time docdf.title.loc[:] = docdf.title.map(nlp)
%time docdf.abstract.loc[:] = docdf.abstract.map(nlp)

CPU times: user 2min 11s, sys: 1.35 s, total: 2min 12s
Wall time: 1min 9s
CPU times: user 8min 14s, sys: 49.3 s, total: 9min 3s
Wall time: 4min 35s


In [179]:
docdf.head()

,id,created,setspec,title,abstract
0,1707.06238,2017-07-19,physics:astro-ph,"(Orbiting, a, binary, :, SPHERE, characterisat...","( , In, this, paper, we, present, the, result..."
1,1707.06239,2017-07-19,physics:astro-ph,"(An, astrometric, search, method, for, individ...","( , Gravitational, waves, (, GWs, ), cause, t..."
2,1707.06240,2017-07-19,math,"(Second, -, order, Bounds, of, Gaussian, Kerne...","( , Guaranteeing, stability, of, a, designed,..."
3,1707.06241,2017-07-19,physics:astro-ph,"(MUSE, sneaks, a, peek, at, extreme, ram, -, p...","( , We, present, the, MUSE, observations, of,..."
4,1707.06242,2017-07-19,physics:hep-th,"(Green, -, Schwarz, Automorphisms, and, 6D, SC...","( , All, known, interacting, 6D, superconform..."


In [197]:
def sim_df(docdf):
    d = defaultdict(list)
    for (index_1, row_1), (index_2, row_2) in product(docdf.iterrows(), repeat=2):
        d[row_1.id].append(row_1.title.similarity(row_2.title))

    min_sim = pd.DataFrame(data=d, index=docdf.id, columns=docdf.id)
    return min_sim

In [198]:
%time min_sim = sim_df(docdf.iloc[:1000,:1000])

CPU times: user 45.1 s, sys: 363 ms, total: 45.4 s
Wall time: 45.9 s


In [216]:
min_sim.loc['1707.06238',:].sort_values(ascending=False).head()

id
1707.06238    1.000000
1707.06463    0.901182
1707.06437    0.872049
1611.01794    0.867840
1707.06496    0.866621
Name: 1707.06238, dtype: float64

In [204]:
docdf.title[0].vocab == docdf.abstract[1].vocab

True

In [206]:
v = docdf.title[0].vocab

In [207]:
v.lang

'en'

In [208]:
v.length

58707